In [7]:
import pandas as pd
import torch
import pickle
from ast import literal_eval
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px
import plotly.io as pio
from sklearn.metrics import mean_absolute_error

In [8]:
from SimplifiedLatentDecoderRing import LatentDecoderRing

In [9]:
model_path = '/Users/HesamFallahian/Sources/cgan-eval/SavedModels/WAE_0043_Latent_model_at_epoch_100.pth'
latent_decoder_ring = LatentDecoderRing(model_path)

In [10]:
column_names = [f'latent_{i}' for i in range(47)] 

In [6]:
file_names_5 = ["result_4p4_5_07022024.pkl", "result_4p6_5_07022024.pkl", "result_5p2_5_07022024.pkl",
              "result_6p4_5_07022024.pkl", "result_7p2_5_07022024.pkl", "result_7p8_5_07022024.pkl",
              "result_8p4_5_07022024.pkl", "result_10p4_5_07022024.pkl", "result_11p4_5_07022024.pkl"]

file_names_3 = ["result_3p6_3_07022024.pkl", "result_4p4_3_07022024.pkl", "result_4p6_3_07022024.pkl", "result_5p2_3_07022024.pkl",
              "result_6p4_3_07022024.pkl", "result_7p2_3_07022024.pkl", "result_7p8_3_07022024.pkl",
              "result_8p4_3_07022024.pkl", "result_10p4_3_07022024.pkl", "result_11p4_3_07022024.pkl"]

time_interval = 3

In [ ]:
for file_name in file_names_3:  
    dataframes = []
    
    df_result = pd.read_pickle(f"EvaluationResult/{time_interval}/{file_name}.pkl")
    for i in range(5, 1201, 5):
        file_path = f"Data/3p6/latent_representation_for_{i}.pkl"
        df_res = df_result[df_result["time"] == i]
        df_p = pd.read_pickle(file_path)
        df_p = pd.merge(df_p, df_res[['x', 'y', 'z']], how='inner', on=['x', 'y', 'z'])
        
        df_res['latent_all'] = list(zip(*[df_res[name] for name in column_names]))
        df_p['latent_all'] = list(zip(*[df_p[name] for name in column_names]))
        
        df_res['original'] = df_res['latent_all'].apply(lambda x: latent_decoder_ring.Centroid_Only_DeNormalized(torch.tensor(x))) 
        df_p['original'] = df_p['latent_all'].apply(lambda x: latent_decoder_ring.Centroid_Only_DeNormalized(torch.tensor(x))) 
        
        df_res[['vx', 'vy', 'vz']] = pd.DataFrame(df_res['original'].tolist(), index=df_res.index)
        df_p[['vx', 'vy', 'vz']] = pd.DataFrame(df_p['original'].tolist(), index=df_p.index)
        
        df_p.drop(columns = df_p.columns.difference(['x','y','z','vx','vy','vz']), axis=1, inplace=True)
        df_res.drop(columns = df_res.columns.difference(['x','y','z','vx','vy','vz']), axis=1, inplace=True)
        
        df_p.loc[:, "vx"]= df_p["vx"]/10
        df_p.loc[:, "vy"]= df_p["vy"]/10
        df_p.loc[:, "vz"]= df_p["vz"]/10
        df_res.loc[:, "vx"]= df_res["vx"]/10
        df_res.loc[:, "vy"] = df_res["vy"]/10
        df_res.loc[:, "vz"]= df_res["vz"]/10
        
        merged_df = pd.merge(df_p, df_res, how='inner', left_on=['x', 'y', 'z'], right_on=['x', 'y', 'z'])
        merged_df['vx_error'] = abs(merged_df['vx_x'] - merged_df['vx_y'])
        merged_df['vy_error'] = abs(merged_df['vy_x'] - merged_df['vy_y'])
        merged_df['vz_error'] = abs(merged_df['vz_x'] - merged_df['vz_y'])
        merged_df['time'] = i
        dataframes.append(merged_df)

In [8]:
combined_df = pd.concat(dataframes)

In [12]:
combined_df.head()

,x,y,z,vx_x,vy_x,vz_x,vx_y,vy_y,vz_y,vx_error,vy_error,vz_error,time
0,-117.0,71.0,-25.0,0.048930,0.004642,-0.000857,0.048976,0.004686,-0.000800,0.000046,0.000045,0.000057,5
1,-97.0,71.0,-25.0,0.049188,0.004064,0.001669,0.049244,0.004076,0.001700,0.000057,0.000012,0.000031,5
2,-78.0,71.0,-25.0,0.049293,0.003150,0.001643,0.049233,0.003042,0.001595,0.000060,0.000108,0.000048,5
3,-58.0,71.0,-25.0,0.050413,0.002313,0.003913,0.050366,0.002316,0.003880,0.000047,0.000003,0.000034,5
4,-38.0,71.0,-25.0,0.048525,0.003128,0.003365,0.048500,0.003084,0.003415,0.000026,0.000044,0.000049,5


In [13]:
combined_df.to_pickle("Post/3p6.pkl")

In [11]:
dataframes = []
    
df_result = pd.read_pickle(f"EvaluationResult/3/result_3p6_3_07022024.pkl")
for i in range(3, 1201, 3):
    file_path = f"Data/3p6/latent_representation_for_{i}.pkl"
    df_res = df_result[df_result["time"] == i]
    df_p = pd.read_pickle(file_path)
    df_p = pd.merge(df_p, df_res[['x', 'y', 'z']], how='inner', on=['x', 'y', 'z'])
    
    df_res['latent_all'] = list(zip(*[df_res[name] for name in column_names]))
    df_p['latent_all'] = list(zip(*[df_p[name] for name in column_names]))
    
    df_res['original'] = df_res['latent_all'].apply(lambda x: latent_decoder_ring.Centroid_Only_DeNormalized(torch.tensor(x))) 
    df_p['original'] = df_p['latent_all'].apply(lambda x: latent_decoder_ring.Centroid_Only_DeNormalized(torch.tensor(x))) 
    
    df_res[['vx', 'vy', 'vz']] = pd.DataFrame(df_res['original'].tolist(), index=df_res.index)
    df_p[['vx', 'vy', 'vz']] = pd.DataFrame(df_p['original'].tolist(), index=df_p.index)
    
    df_p.drop(columns = df_p.columns.difference(['x','y','z','vx','vy','vz']), axis=1, inplace=True)
    df_res.drop(columns = df_res.columns.difference(['x','y','z','vx','vy','vz']), axis=1, inplace=True)
    
    df_p.loc[:, "vx"]= df_p["vx"]/10
    df_p.loc[:, "vy"]= df_p["vy"]/10
    df_p.loc[:, "vz"]= df_p["vz"]/10
    df_res.loc[:, "vx"]= df_res["vx"]/10
    df_res.loc[:, "vy"] = df_res["vy"]/10
    df_res.loc[:, "vz"]= df_res["vz"]/10
    
    merged_df = pd.merge(df_p, df_res, how='inner', left_on=['x', 'y', 'z'], right_on=['x', 'y', 'z'])
    merged_df['vx_error'] = abs(merged_df['vx_x'] - merged_df['vx_y'])
    merged_df['vy_error'] = abs(merged_df['vy_x'] - merged_df['vy_y'])
    merged_df['vz_error'] = abs(merged_df['vz_x'] - merged_df['vz_y'])
    merged_df['time'] = i
    dataframes.append(merged_df)combined_df = pd.concat(dataframes)

/var/folders/d_/7d_hplr504q05f701wws0w9m0000gp/T/ipykernel_75563/2954489218.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_res['latent_all'] = list(zip(*[df_res[name] for name in column_names]))
/var/folders/d_/7d_hplr504q05f701wws0w9m0000gp/T/ipykernel_75563/2954489218.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_res['original'] = df_res['latent_all'].apply(lambda x: latent_decoder_ring.Centroid_Only_DeNormalized(torch.tensor(x)))
/var/folders/d_/7d_hplr504q05f701wws0w9m0000gp/T/ipykern

FileNotFoundError: [Errno 2] No such file or directory: 'Data/3p6/latent_representation_for_1173.pkl'

In [12]:
combined_df = pd.concat(dataframes)

In [13]:
combined_df.head()

,x,y,z,vx_x,vy_x,vz_x,vx_y,vy_y,vz_y,vx_error,vy_error,vz_error,time
0,-117.0,71.0,-25.0,0.048021,0.004009,0.003618,0.048050,0.003963,0.003575,0.000029,0.000046,0.000043,3
1,-97.0,71.0,-25.0,0.049069,0.003970,0.000910,0.049103,0.003962,0.000943,0.000035,0.000008,0.000034,3
2,-78.0,71.0,-25.0,0.049551,0.003615,0.003332,0.049605,0.003605,0.003352,0.000054,0.000010,0.000020,3
3,-58.0,71.0,-25.0,0.049275,0.002748,0.002407,0.049222,0.002726,0.002377,0.000053,0.000022,0.000030,3
4,-38.0,71.0,-25.0,0.048799,0.002499,0.002000,0.048758,0.002440,0.001999,0.000040,0.000059,0.000001,3


In [14]:
combined_df["time"].value_counts()

time
3       5290
735     5290
801     5290
798     5290
795     5290
        ... 
384     5290
381     5290
378     5290
375     5290
1170    5290
Name: count, Length: 390, dtype: int64

In [15]:
combined_df.to_pickle("Post/3p6_3.pkl")